---
title: Accessing and visualizing data from the VEDA Data Catalog
---

VEDA uses a Spatio-Temporal Asset Catalog (STAC) to catalog its listing of datasets.
This tutorial teaches the basics of searching and accessing datasets using the catalog and visualizing the data in a map.

## Approach
- Use the `pystac_client` library to connect to the data catalog
- List all the collections from the catalog
- Look at the items in one of the collections
- Access an asset in the item
- Use the VEDA TiTiler API to visualize the asset in a map
- Perform a basic search of the catalog

## Libraries Used

### PySTAC Client

From the PySTAC Client Documentation:

> The STAC Python Client (`pystac_client`) is a Python package for working with STAC Catalogs and APIs that conform to the STAC and STAC API specs in a seamless way. \
> PySTAC Client builds upon PySTAC through higher-level functionality and ability to leverage STAC API search endpoints.

We will use it to interact with the data catalog.

### Requests

Requests (`requests`) is a simple HTTP library. Requests allows you to send HTTP requests extremely easily. 

We will use it to make API requests.

### Leafmap

A Python package for geospatial analysis and interactive mapping in a Jupyter environment.

In [76]:
pip install leafmap --quiet

Note: you may need to restart the kernel to use updated packages.


Here we import all the required libraries and modules.

In [77]:
import requests
import leafmap

from pystac_client import Client

# For displaying image in a jupyter notebook
from IPython.display import Image, display

## Accessing data from a STAC Catalog

Let's define the data catalog (STAC catalog) API URL

In [78]:
# STAC API root URL
URL = 'https://staging-stac.delta-backend.com'

Use the `pystac_client.Client` module to connect to the catalog

In [79]:
catalog = Client.open(URL)
catalog

<Client id=stac-fastapi>

List all the datasets (collections) in the catalog

In [80]:
collections = catalog.get_collections()
collections = list(collections)

In [81]:
for collection in collections:
    # print(f">> {collection.id}: \n- {collection.description}\n")
    
    print(">> " + collection.id)
    print("- " + collection.description + "\n")

>> campfire-lst-day-diff
- MODIS WSA Albedo difference from a three-year average of 2015 to 2018 subtracted from a three-year average of 2019-2022. These tri-annual averages represent periods before and after the fire.

>> grdi-vnl-slope-raster
- Global Gridded Relative Deprivation Index (GRDI) VIIRS Night Lights (VNL) Slope Constituent raster

>> nightlights-500m-daily
- Darker colors indicate fewer night lights and less activity. Lighter colors indicate more night lights and more activity. Check out the HD dataset to see a light-corrected version of this dataset.

>> social-vulnerability-index-household
- Household Composition & Disability (Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households) - Percentile ranking

>> mtbs-burn-severity
- Burn severities and extents of fires from Monitoring Trends in Burn Severity (MTBS) program during the years 2016-2020 over Western US

>> ecco-surface-height-change
- Gridded global sea-surface height change fr

Pick a collection to interact with. Let's pick `no2-monthly`. This dataset represents the average monthly Nitrogen Dioxide emission.

Read more about the dataset in the VEDA Web Portal: https://www.earthdata.nasa.gov/dashboard/data-catalog/no2

In [82]:
collection = catalog.get_collection("no2-monthly")
collection

<CollectionClient id=no2-monthly>

Let's look at the items within the collection, using the `CollectionClient.get_items` method

In [ ]:
# get_items
items = list(collection.get_items())
items[-1]

In [ ]:
for item in items[:10]:
    print(item)

In [ ]:
item = collection.get_item(items[0].id)   # item = collection.get_item("finalBMHD_ScaledVenice_202203")
item

In [ ]:
item = item.to_dict()
asset = item["assets"]["cog_default"]["href"]
asset

## Visualization using TiTiler

In [ ]:
# Define the TiTiler url
TITILER_URL = "https://staging-raster.delta-backend.com"

Now we extract the visualization metadata for this collection. This defines how the data is shown in the map - what colors are used, how do we wanna rescale the values, what resampling method to use, etc. 

In [ ]:
# Extract visualization metadata for items
collection_dict = collection.to_dict()
viz_meta = collection_dict["renders"]["dashboard"]
viz_meta

In [ ]:
# Make request to TiTiler for tiles for the specific item using `/stac/tilejson.json` endpoint

tiles = requests.get(
    f"{TITILER_URL}/stac/tilejson.json?collection={item['collection']}&item={item['id']}"
    f"&assets={viz_meta['assets'][0]}"
    f"&colormap_name={viz_meta['colormap_name']}"
    f"&rescale={viz_meta['rescale'][0][0]},{viz_meta['rescale'][0][1]}",
    # f"&rescale=0,6000000000000000",
).json()
tiles

User `requests.get` to make a `GET` request for the preview

In [ ]:
map = leafmap.Map(center=tiles["center"], zoom=2, draw_control=False, measure_control=False, 
                  fullscreen_control=False, attribution_control=True)
map.add_tile_layer(tiles["tiles"][0], name=collection_dict["title"], attribution=collection_dict["providers"][0]["name"])
map

#### Searching for datasets based on the Area of Interest (AOI) and/or datetime

In [ ]:
# Rough AOI for India
india_aoi = {
        "coordinates": [
          [
            80.06099189181373,
            28.8399703979607
          ],
          [
            80.73279011182046,
            30.45698376236544
          ],
          [
            76.67531159236876,
            33.23515362865537
          ],
          [
            74.28488701347192,
            31.043748244813315
          ],
          [
            69.77872122951806,
            25.779691430137802
          ],
          [
            68.4716652377574,
            22.838387811208662
          ],
          [
            77.04776242546541,
            7.788680292842955
          ],
          [
            78.17658261684147,
            7.8872825678783585
          ],
          [
            81.11174176932082,
            12.496703434512142
          ],
          [
            80.6847796861473,
            15.35801808733666
          ],
          [
            88.98770999400523,
            21.57078959559047
          ],
          [
            88.09195930859318,
            26.374746793870216
          ],
          [
            83.26895036383235,
            26.942804829426848
          ],
          [
            79.87153979372539,
            28.800935184841933
          ],
          [
            80.5901083982061,
            30.50111494156242
          ]
        ],
        "type": "LineString"
      }


In [ ]:
# Search the catalog for the given collection, aoi and datetime
search = catalog.search(
    max_items = 100,
    # limit = 5,
    collections = "no2-monthly",
    intersects = india_aoi,
    datetime = "2020-01-01/2022-12-31",
)
items = list(search.item_collection())

len(items)

In [ ]:
items[:5]

## Summary

In this notebook, we used the `pystac_client` library to interact with the VEDA Data Catalog. We learned how to list all the datasets, list all the items in a dataset, read the metadata for the dataset/items, access the data file, visualize it and finally search the catalog for specific area of interest and datetime.